# Setup

In [1]:
# Impot modules
import os
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from imblearn.over_sampling import SMOTENC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt 

# Suppress warnings
import warnings
warnings.filterwarnings("ignore") 

# Disable interactive mode
plt.ioff()

In [2]:
# Create/Map directories
if not os.path.exists('result'): os.mkdir('result')
data_path = os.path.join(os.getcwd(), "data")
result_path = os.path.join(os.getcwd(), "result")

# Import

In [3]:
sheet_to_df_map = pd.read_excel(os.path.join(data_path, "full.xlsx"), sheet_name=None)
full = pd.concat(sheet_to_df_map, axis=0, ignore_index=True)

recategorised_data = pd.read_csv(os.path.join(data_path, "recategorised_data.csv"))

# Tidy

In [4]:
# Raw data for exploration
full.dropna()
full["Class"] = np.where(full["Entero"]>=280, 1, 0)
full = full.drop(["Entero"], axis=1)

# Recategorsied data for modelling
recategorised_data = recategorised_data.drop(["Entero", "RainWA", "BeachName", "Wdirection_level", "Wspeed_level", "BeachDirection"], axis=1)

# Converted to datatime64 for ordering
recategorised_data["DATE"] = recategorised_data["DATE"].astype("datetime64")

# Converted to catogory for oversampling
recategorised_data["BeachType"] = recategorised_data["BeachType"].astype("category")
recategorised_data["on_offshore"] = recategorised_data["on_offshore"].astype("category")

# Scale down the categorical features to (0, 1)
recategorised_data["on_offshore"] = recategorised_data["on_offshore"].apply(lambda x: x/2)

# Rearrange the features, so numerics comes first for nomalisation
recategorised_data = recategorised_data.reindex(columns=(list([c for c in recategorised_data.columns if c != "Entero_level"]) + ["Entero_level"]))

# Exploration

In [5]:
# Raw data
full

,Date,Rain24,Rain48,Rain72,RainWA,WDirection,WSpeed,SolarHours,Class
0,1995-12-16,9.17,13.67,37.00,59.84,200,1.5,1.8,0
1,1995-12-27,0.00,0.00,0.33,0.33,230,5.7,8.5,0
2,1995-12-28,2.50,2.50,2.50,7.50,320,3.6,11.7,0
3,1996-01-07,0.00,0.00,0.00,0.00,130,4.1,7.9,0
4,1996-01-08,0.00,0.00,0.00,0.00,100,3.6,4.4,0
...,...,...,...,...,...,...,...,...,...
2014,2017-08-18,1.50,1.50,1.50,4.50,172,1.7,1.6,0
2015,2017-08-28,6.50,8.00,8.00,22.50,5,4.1,3.7,0
2016,2017-09-14,0.00,1.00,4.00,5.00,261,3.1,2.8,0
2017,2017-09-18,26.00,26.50,34.00,86.50,276,2.8,6.1,0


In [6]:
# Stats
stats = full.describe()
stats.to_csv(os.path.join(result_path, "stats.csv"))

In [7]:
# Scatter plots
plt.figure()
figure = sns.pairplot(full, hue="Class").figure
figure.set_size_inches(16, 10)
figure.savefig(os.path.join(result_path, "scatter_plots"), bbox_inches="tight")

In [8]:
# Correlation heatmap
plt.figure()
figure = sns.heatmap(full.corr()).get_figure()
figure.set_size_inches(16, 10)
figure.savefig(os.path.join(result_path, "correlation_heatmap"), bbox_inches="tight")

# Modelling

## Helpers

In [9]:
# Set random state
RANDOM_STATE = 1234
np.random.seed(RANDOM_STATE)

# Order data by date
recategorised_data = recategorised_data.sort_values(by=['DATE'])
recategorised_data = recategorised_data.drop("DATE", axis=1)

# Extract features and label
def extract(dataframe):
    X = dataframe.iloc[:, 0:-1]
    y = dataframe.iloc[:, -1]
    return X, y

# Set test size
test_size = 200

# Hold out the test set
cv_dataset = recategorised_data.iloc[:-test_size, :]

# Timeseries splits for the training
n_split = 5
ts = TimeSeriesSplit(n_splits=n_split)

# Set the number of observations at each step for the test
n_obsn = 40

# Dataframes to store model performances for autorank
dfSen = pd.DataFrame(columns=["KNN", "BDT", "ANN"])
dfAcc = pd.DataFrame(columns=["KNN", "BDT", "ANN"])
dfSp = pd.DataFrame(columns=["KNN", "BDT", "ANN"])

In [10]:
# Training phase to tune hyperparameters
def rolling_cv(hyperparameters, setup_classifier):
    # Pre-allocate space for results
    tscv = np.zeros((ts.n_splits, len(hyperparameters)))

    # Multi-split the data into train sets and validation sets in a timely manner
    ts_idx = -1
    for train_index, validation_index in ts.split(cv_dataset):    
        ts_idx += 1
        train, validation = cv_dataset.iloc[train_index, :], cv_dataset.iloc[validation_index, :]
        
        X_train, y_train = extract(train)
        X_valdn, y_valdn = extract(validation)
        
        # Fit the scaler to X_train, and then use it to transform both the train set and the test set
        transfromer = preprocessing.Normalizer().fit(X_train.iloc[:, 0:6])
        X_train.iloc[:, 0:6] = transfromer.transform(X_train.iloc[:, 0:6])
        X_valdn.iloc[:, 0:6] = transfromer.transform(X_valdn.iloc[:, 0:6])

        # Oversample the train set with SMOTENC
        smotenc = SMOTENC(categorical_features=[X_train.dtypes=="category"], sampling_strategy="minority", k_neighbors=1)
        X_train, y_train = smotenc.fit_resample(X_train, y_train)

        # Test hyperparameters
        idx = -1
        for i in hyperparameters:
            idx += 1
            classifier = setup_classifier(i)
            fitted_model = classifier.fit(X_train, y_train)
            
            # Sensitivity as the optimisation target
            tn, fp, fn, tp = confusion_matrix(y_valdn, fitted_model.predict(X_valdn)).ravel()
            tscv[ts_idx, idx] = tp/(tp+fn)
            
    return tscv

In [11]:
# Testing phase for the performances from the best models, i.e. KNN, BDT, ANN
def walking_eval(classifier, classifier_name, l_test=test_size, step=n_obsn):
    sen = []
    sp = []
    acc = []
    
    for i in range(0, l_test, n_obsn):
        if l_test-i < n_obsn: break
        train = recategorised_data.iloc[0:-(l_test-i), :]
        test = recategorised_data.iloc[-(l_test-i):-(l_test-i-n_obsn if l_test-i-n_obsn!=0 else 1), :]
        
        X_train, y_train = extract(train)
        X_test, y_test = extract(test)
        
        transfromer = preprocessing.Normalizer().fit(X_train.iloc[:, 0:6])
        X_train.iloc[:, 0:6] = transfromer.transform(X_train.iloc[:, 0:6])
        X_test.iloc[:, 0:6] = transfromer.transform(X_test.iloc[:, 0:6])

        smotenc = SMOTENC(categorical_features=[X_train.dtypes=="category"], sampling_strategy="minority", k_neighbors=1)
        X_train, y_train = smotenc.fit_resample(X_train, y_train)

        fitted_model = classifier.fit(X_train, y_train)
        
        tn, fp, fn, tp = confusion_matrix(y_test, fitted_model.predict(X_test)).ravel()
        sen.append(tp/(tp+fn))
        acc.append((tp+tn)/(tn+fp+fn+tp))
        sp.append(tn/(tn+fp))
        
        
    dfSen.loc[:, classifier_name] = sen
    dfAcc.loc[:, classifier_name] = acc
    dfSp.loc[:, classifier_name] = sp
    
    return np.array(sen)

## KNN

In [12]:
# Train
neighbors = range(1, 41, 1)
def knn(i):
    knn = KNeighborsClassifier(n_neighbors=i)
    return knn
cv = rolling_cv(neighbors, knn)
means = np.mean(cv, axis=0)
best = neighbors[np.argmax(means)]
print("The best KNN is with %s neighbour(s)" %(best))

# Evaluate
r = walking_eval(knn(best), "KNN")
print("The average sensitivity for the best KNNs is %s" %(np.mean(r)))

The best KNN is with 39 neighbour(s)
The average sensitivity for the best KNNs is 0.6428571428571429


## BDT

In [ ]:
# Train
n_estimators = range(1, 101, 1)
def ada(i):
    ada = AdaBoostClassifier(n_estimators=i)
    return ada
cv = rolling_cv(n_estimators, ada)
means = np.mean(cv, axis=0)
best = n_estimators[np.argmax(means)]
print("The best BDT is with an n_estimators of %s" %(best))

# Evaluate
r = walking_eval(ada(best), "BDT")
print("The average sensitivity for the best BDTs is %s" %(np.mean(r)))

## ANN

In [ ]:
# Skip the training part as the data set is not large enough to tune the hyperparameters properly for ANN

# Evaluate ANN on an adaptive learning rate
ann = MLPClassifier(solver="sgd", learning_rate="adaptive", hidden_layer_sizes=(4, 2), max_iter=1000) 
r = walking_eval(ann, "ANN")
print("The average sensitivity for the ANNs is %s" %(np.mean(r)))

# Conclusion

In [ ]:
stats = dfSen.describe()
stats.to_csv(os.path.join(result_path, "sensitivity_stats.csv"))
figure = plt.figure()
dfSen.boxplot()
figure.set_size_inches(16, 10)
figure.savefig(os.path.join(result_path, "sensitivity_results"), bbox_inches="tight")

In [ ]:
stats = dfAcc.describe()
stats.to_csv(os.path.join(result_path, "accuracy_stats.csv"))
figure = plt.figure()
dfAcc.boxplot()
figure.set_size_inches(16, 10)
figure.savefig(os.path.join(result_path, "accuracy_results"), bbox_inches="tight")

In [ ]:
stats = dfSp.describe()
stats.to_csv(os.path.join(result_path, "specificity_stats.csv"))
figure = plt.figure()
dfSp.boxplot()
figure.set_size_inches(16, 10)
figure.savefig(os.path.join(result_path, "specificity_results"), bbox_inches="tight")